In [1]:
from dotenv import load_dotenv
import base64
from requests import post, get
import json
import os
import urllib.request
import requests
import pandas as pd
from tqdm import tqdm

In [2]:
client_id = 'ee1fcd3c27364f18b6a6ac52a24017a0'
client_secret = '408b3a91cf844bf8a37bde42901c7445'

def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

In [230]:
#access_token = get_token()

In [231]:
access_token = 'BQB1-sfTD8v44F5tvO_y7ATTl3n3fVWJxB9r1iHPLBjBlMMNUofw903bPEoC0FY6Z6BVfFCfjlVpoZ-k4J56_1stSE2yQE4qjPMpuzgsWHyaYIVItZY'

In [232]:
access_token

'BQB1-sfTD8v44F5tvO_y7ATTl3n3fVWJxB9r1iHPLBjBlMMNUofw903bPEoC0FY6Z6BVfFCfjlVpoZ-k4J56_1stSE2yQE4qjPMpuzgsWHyaYIVItZY'

In [24]:
def find_playlist_id(access_token, Country):
    playlist_name = f'Top 50 - {Country}'
    query = urllib.parse.quote(playlist_name)

    url = f"https://api.spotify.com/v1/search?q={query}&type=playlist&market=US&limit=10"

    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        playlists = response.json()['playlists']['items']

        for playlist in playlists:
            if playlist['name'].lower() == playlist_name.lower():
                return playlist['id']

In [25]:
def get_playlist_tracks(access_token, playlist_id):
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"

    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        tracks = response.json()['items']
        tracks_set = {}
        for item in tracks:
            track = item['track']
            tracks_set[track['name']] = track['id']
        return tracks_set

In [26]:
def get_track_audio_features(access_token, track_id):

    url = f"https://api.spotify.com/v1/audio-features/{track_id}"

    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }

    response = requests.get(url, headers=headers)
    features_dict = {}
    if response.status_code == 200:
        audio_features = response.json()
        list_features = []
        duration = audio_features['duration_ms']
        loud = audio_features['loudness']
        temp = audio_features['tempo']
        energy = audio_features['energy']
        dance = audio_features['danceability']
        acoustic = audio_features['acousticness']
        instrument = audio_features['instrumentalness']
        live = audio_features['liveness']
        valence = audio_features['valence']
        speech = audio_features['speechiness']
        major_minor = "Major" if audio_features['mode'] == 1 else "Minor"
        list_features += [duration, loud, temp, energy, dance, acoustic, instrument, live, valence, speech, major_minor]
        return list_features
#    else:
#        return ["None"] * 11

In [27]:
df = pd.read_excel("C:/Users/Egor/PycharmProjects/Spotify API/Charts.xlsx")['TOP-50']
countries_list = df.tolist()
my_dict = {}

In [46]:
for value in tqdm(countries_list, desc="Processing countries"):
    my_dict[value] = find_playlist_id(access_token, value)

Processing countries: 100%|████████████████████████████████████████████████████████████| 76/76 [00:17<00:00,  4.40it/s]


In [47]:
my_dict

{'Albania': None,
 'Argentina': '37i9dQZEVXbMMy2roB9myp',
 'Australia': '37i9dQZEVXbJPcfkRz0wJ0',
 'Austria': '37i9dQZEVXbKNHh6NIXu36',
 'Belarus': '37i9dQZEVXbIYfjSLbWr4V',
 'Belgium': '37i9dQZEVXbJNSeeHswcKB',
 'Bolivia': '37i9dQZEVXbJqfMFK4d691',
 'Brazil': '37i9dQZEVXbMXbN3EUUhlg',
 'Bulgaria': '37i9dQZEVXbNfM2w2mq1B8',
 'Canada': '37i9dQZEVXbKj23U1GF4IR',
 'Chile': '37i9dQZEVXbL0GavIqMTeb',
 'Colombia': '37i9dQZEVXbOa2lmxNORXQ',
 'Costa Rica': '37i9dQZEVXbMZAjGMynsQX',
 'Cyprus': '1NNauN205jiVeH2ixReshi',
 'Czech Republic': '37i9dQZEVXbIP3c3fqVrJY',
 'Denmark': '37i9dQZEVXbL3J0k32lWnN',
 'Dominican Republic': '37i9dQZEVXbKAbrMR8uuf7',
 'Ecuador': '37i9dQZEVXbJlM6nvL1nD1',
 'Egypt': '37i9dQZEVXbLn7RQmT5Xv2',
 'El Salvador': '37i9dQZEVXbLxoIml4MYkT',
 'Estonia': '37i9dQZEVXbLesry2Qw2xS',
 'Finland': '37i9dQZEVXbMxcczTSoGwZ',
 'France': '37i9dQZEVXbIPWwFssbupI',
 'Germany': '37i9dQZEVXbJiZcmkrIHGU',
 'Greece': '37i9dQZEVXbJqdarpmTJDL',
 'Guatemala': '37i9dQZEVXbLy5tBFyQvd4',
 'Hondur

In [57]:
for key in tqdm(my_dict, desc="Processing countries"):
    my_dict[key] = get_playlist_tracks(access_token, my_dict[key])

Processing countries: 100%|████████████████████████████████████████████████████████████| 76/76 [01:00<00:00,  1.25it/s]


In [58]:
my_dict

{'Albania': None,
 'Argentina': {'BESAME (feat. Tiago PZK, Khea & Neo Pistea) - Remix': '3GD6eImRvT0zgr8cQnokUq',
  'Un Besito Más': '5rQSQlZXXjMcevPGoAfE1z',
  'Piel': '4wS0TnQzVkY9ML1BPKpOk1',
  '30 GRADOS': '0GVPemmAwkXhFlYimhdDr3',
  'LUNA': '7bywjHOc0wSjGGbj04XbVi',
  'HOLA PERDIDA': '7JKptWv4YXmievid0HajID',
  'Tal Para Cual': '0mJRX0WnqWlu8Boe7gpQ1P',
  'Santa': '5bi0gh89wRuH2OgjdAKFsb',
  'UH LA LA LA': '2eV3pmYGM1bm1vWHc0t8WF',
  'La_Original.mp3': '6Es8Sk3xe1HiJ2MXCfHHwR',
  'Gata Only': '6XjDF6nds4DE2BBbagZol6',
  'Ojos Verdes': '2RmtEc2ZNDCzAU3vH0AbKx',
  'Exclusive.mp3': '5f4mOETLngamMDTJnbF9s7',
  'GAFITA Y CHOMBA': '1905loqmX0pSFPMbA9rc1f',
  'BBY BOO - REMIX': '43PiEypaceH339CSNiggyj',
  'Una Foto Remix (feat. Emilia)': '1kqH58eGh2ZTOHwqBIB2tM',
  'Tumba': '5Ywg69AnG1CLLGEXne1MVy',
  'FLECHAZO EN EL CENTRO': '4jn3R3KF5hx2fCiXlj89yE',
  'Mil Preguntas': '42mrOMjkQb7jRKatu0bqg4',
  'QUE ME FALTE TODO': '1TyBUtMxdmRfXhW3MmbxtQ',
  'offline': '7s4emXHJwAuo5VvGBXmnBL',
  'M.

In [59]:
df = pd.DataFrame()
df['Страны'] = None
df['Название трека'] = None
df['ID трека'] = None
for country, songs in tqdm(my_dict.items(), desc="Processing countries"):
    if songs is not None:
        for song, song_id in songs.items():
            new_row = pd.DataFrame({'Страны': [country], 'Название трека': [song], 'ID трека': [song_id]})
            df = pd.concat([df, new_row], ignore_index=True)

Processing countries: 100%|████████████████████████████████████████████████████████████| 76/76 [00:01<00:00, 58.76it/s]


In [60]:
df

,Страны,Название трека,ID трека
0,Argentina,"BESAME (feat. Tiago PZK, Khea & Neo Pistea) - ...",3GD6eImRvT0zgr8cQnokUq
1,Argentina,Un Besito Más,5rQSQlZXXjMcevPGoAfE1z
2,Argentina,Piel,4wS0TnQzVkY9ML1BPKpOk1
3,Argentina,30 GRADOS,0GVPemmAwkXhFlYimhdDr3
4,Argentina,LUNA,7bywjHOc0wSjGGbj04XbVi
...,...,...,...
3555,Vietnam,Anh Là Ngoại Lệ Của Em,6KgZf5gZgF1KLNxq1qZgHP
3556,Vietnam,bao tiền một mớ bình yên?,7kgzRBNkBRArtTPkYjWLYT
3557,Vietnam,đưa em về nhàa,5LQpQ6SDeaz6SIkR0lvsoC
3558,Vietnam,Ánh Sao Và Bầu Trời,5iZHnufFUgATzrpGgH1K0W


In [61]:
df = df.rename(columns={'Название трека': 'Трек'})

In [62]:
df['ID трека']

0       3GD6eImRvT0zgr8cQnokUq
1       5rQSQlZXXjMcevPGoAfE1z
2       4wS0TnQzVkY9ML1BPKpOk1
3       0GVPemmAwkXhFlYimhdDr3
4       7bywjHOc0wSjGGbj04XbVi
                 ...          
3555    6KgZf5gZgF1KLNxq1qZgHP
3556    7kgzRBNkBRArtTPkYjWLYT
3557    5LQpQ6SDeaz6SIkR0lvsoC
3558    5iZHnufFUgATzrpGgH1K0W
3559    2doGmsaPLdot51o80juiYU
Name: ID трека, Length: 3560, dtype: object

In [63]:
df['Длительность'] = None
df['Громкость'] = None
df['Тепм'] = None
df['Энергичность'] = None
df['Танцевальность'] = None
df['Акустичность'] = None
df['Инструментальность'] = None
df['Живость'] = None
df['Валентность'] = None
df['Речевой контент'] = None
df['Мажорность/минорность'] = None


In [64]:
df

,Страны,Трек,ID трека,Длительность,Громкость,Тепм,Энергичность,Танцевальность,Акустичность,Инструментальность,Живость,Валентность,Речевой контент,Мажорность/минорность
0,Argentina,"BESAME (feat. Tiago PZK, Khea & Neo Pistea) - ...",3GD6eImRvT0zgr8cQnokUq,None,None,None,None,None,None,None,None,None,None,None
1,Argentina,Un Besito Más,5rQSQlZXXjMcevPGoAfE1z,None,None,None,None,None,None,None,None,None,None,None
2,Argentina,Piel,4wS0TnQzVkY9ML1BPKpOk1,None,None,None,None,None,None,None,None,None,None,None
3,Argentina,30 GRADOS,0GVPemmAwkXhFlYimhdDr3,None,None,None,None,None,None,None,None,None,None,None
4,Argentina,LUNA,7bywjHOc0wSjGGbj04XbVi,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3555,Vietnam,Anh Là Ngoại Lệ Của Em,6KgZf5gZgF1KLNxq1qZgHP,None,None,None,None,None,None,None,None,None,None,None
3556,Vietnam,bao tiền một mớ bình yên?,7kgzRBNkBRArtTPkYjWLYT,None,None,None,None,None,None,None,None,None,None,None
3557,Vietnam,đưa em về nhàa,5LQpQ6SDeaz6SIkR0lvsoC,None,None,None,None,None,None,None,None,None,None,None
3558,Vietnam,Ánh Sao Và Bầu Trời,5iZHnufFUgATzrpGgH1K0W,None,None,None,None,None,None,None,None,None,None,None


In [65]:
len(df)

3560

In [83]:
for i in tqdm(df['ID трека'], desc="Processing countries"):
    featur_list = get_track_audio_features(access_token, i)
    a = featur_list
    track_index = df.loc[df['ID трека'] == i].index[0]
    df.loc[track_index, 'Длительность'] = a[0]
    df.loc[track_index, 'Громкость'] = a[1]
    df.loc[track_index, 'Тепм'] = a[2]
    df.loc[track_index, 'Энергичность'] = a[3]
    df.loc[track_index, 'Танцевальность'] = a[4]
    df.loc[track_index, 'Акустичность'] = a[5]
    df.loc[track_index, 'Инструментальность'] = a[6]
    df.loc[track_index, 'Живость'] = a[7]
    df.loc[track_index, 'Валентность'] = a[8]
    df.loc[track_index, 'Речевой контент'] = a[9]
    df.loc[track_index, 'Мажорность/минорность'] = a[10]


    
    

Processing countries:  56%|███████████████████████████████▎                        | 1993/3560 [10:58<08:37,  3.03it/s]


TypeError: 'NoneType' object is not subscriptable

In [165]:
for index in tqdm(range(1991, len(df)), desc="Processing tracks"):
    track_id = df.loc[index, 'ID трека']
    feature_list = get_track_audio_features(access_token, track_id)
    a = feature_list
    df.loc[index, 'Длительность'] = a[0]
    df.loc[index, 'Громкость'] = a[1]
    df.loc[index, 'Темп'] = a[2]
    df.loc[index, 'Энергичность'] = a[3]
    df.loc[index, 'Танцевальность'] = a[4]
    df.loc[index, 'Акустичность'] = a[5]
    df.loc[index, 'Инструментальность'] = a[6]
    df.loc[index, 'Живость'] = a[7]
    df.loc[index, 'Валентность'] = a[8]
    df.loc[index, 'Речевой контент'] = a[9]
    df.loc[index, 'Мажорность/минорность'] = a[10]

Processing tracks: 100%|███████████████████████████████████████████████████████████| 1569/1569 [08:22<00:00,  3.12it/s]


In [168]:
df

,Страны,Трек,ID трека,Длительность,Громкость,Тепм,Энергичность,Танцевальность,Акустичность,Инструментальность,Живость,Валентность,Речевой контент,Мажорность/минорность,Темп
0,Argentina,"BESAME (feat. Tiago PZK, Khea & Neo Pistea) - ...",3GD6eImRvT0zgr8cQnokUq,284308,-6.621,130.104,0.618,0.728,0.203,0,0.127,0.217,0.0673,Major,NaN
1,Argentina,Un Besito Más,5rQSQlZXXjMcevPGoAfE1z,160750,-5.775,92.01,0.641,0.693,0.368,0,0.294,0.514,0.0359,Minor,NaN
2,Argentina,Piel,4wS0TnQzVkY9ML1BPKpOk1,141079,-1.617,168.004,0.907,0.646,0.422,0,0.14,0.816,0.0593,Major,NaN
3,Argentina,30 GRADOS,0GVPemmAwkXhFlYimhdDr3,151441,-3.784,98.325,0.573,0.676,0.0185,0,0.0761,0.369,0.338,Major,NaN
4,Argentina,LUNA,7bywjHOc0wSjGGbj04XbVi,196800,-2.888,100.019,0.86,0.774,0.131,0,0.116,0.446,0.13,Minor,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3555,Vietnam,Anh Là Ngoại Lệ Của Em,6KgZf5gZgF1KLNxq1qZgHP,188108,-5.958,None,0.677,0.844,0.171,0.000023,0.387,0.791,0.0564,Major,111.006
3556,Vietnam,bao tiền một mớ bình yên?,7kgzRBNkBRArtTPkYjWLYT,300000,-10.006,None,0.364,0.429,0.455,0,0.0907,0.0857,0.0547,Major,173.630
3557,Vietnam,đưa em về nhàa,5LQpQ6SDeaz6SIkR0lvsoC,240000,-9.901,None,0.523,0.66,0.714,0.000011,0.357,0.641,0.0476,Major,90.947
3558,Vietnam,Ánh Sao Và Bầu Trời,5iZHnufFUgATzrpGgH1K0W,260571,-14.005,None,0.205,0.662,0.876,0.00354,0.109,0.268,0.0349,Minor,139.911


In [233]:
for index in tqdm(range(0, 1992), desc="Processing tracks"):
    track_id = df.loc[index, 'ID трека']
    feature_list = get_track_audio_features(access_token, track_id)
    a = feature_list
    df.loc[index, 'Длительность'] = a[0]
    df.loc[index, 'Громкость'] = a[1]
    df.loc[index, 'Энергичность'] = a[3]
    df.loc[index, 'Танцевальность'] = a[4]
    df.loc[index, 'Акустичность'] = a[5]
    df.loc[index, 'Инструментальность'] = a[6]
    df.loc[index, 'Живость'] = a[7]
    df.loc[index, 'Валентность'] = a[8]
    df.loc[index, 'Речевой контент'] = a[9]
    df.loc[index, 'Мажорность/минорность'] = a[10]
    df.loc[index, 'Темп'] = a[2]

Processing tracks: 100%|███████████████████████████████████████████████████████████| 1992/1992 [10:46<00:00,  3.08it/s]


In [217]:
df.iloc[422]

Страны                                   Canada
Трек                                   Push Ups
ID трека                 3eh51r6rFWAlGQRlHx9QnQ
Длительность                               None
Громкость                                  None
Тепм                                       None
Энергичность                               None
Танцевальность                             None
Акустичность                               None
Инструментальность                         None
Живость                                    None
Валентность                                None
Речевой контент                            None
Мажорность/минорность                      None
Темп                                        NaN
Name: 422, dtype: object

In [235]:
df.to_excel('C:/Users/Egor/Desktop/андан/dataframe.xlsx', index=False)

In [220]:
df.drop(columns=['Тепм'], inplace=True)